In [18]:
import os
import numpy as np
import pandas as pd
import scipy.io
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence

In [19]:

# Load the data
class ECGDataset(Dataset):
    def __init__(self, mat_files, labels):
        self.mat_files = mat_files
        self.labels = labels

    def __len__(self):
        return len(self.mat_files)

    def __getitem__(self, idx):
        mat_file = self.mat_files[idx]
        label = self.labels[idx]
        
        # Load the .mat file
        ecg_signal = scipy.io.loadmat(mat_file)['val'][0]  
        return torch.tensor(ecg_signal, dtype=torch.float32), label



In [20]:
# Load labels
labels_df = pd.read_csv('C:/Users/elkha/Desktop/EMA/EMA_173_2024-2025/DeepLearning/TP/RNN/Data/labels.csv', header=None, names=['filename', 'label'])
labels_df['filename'] = labels_df['filename'].apply(lambda x: os.path.join('C:/Users/elkha/Desktop/EMA/EMA_173_2024-2025/DeepLearning/TP/RNN/Data/ECG_Data', x))
label_encoder = LabelEncoder()
labels_df['label'] = label_encoder.fit_transform(labels_df['label'])


In [21]:
# Split the dataset into training and test sets
train_files, test_files, train_labels, test_labels = train_test_split(
    labels_df['filename'].values,
    labels_df['label'].values,
    test_size=0.2,
    random_state=42
)

In [22]:
# Create datasets
train_dataset = ECGDataset(train_files, train_labels)
test_dataset = ECGDataset(test_files, test_labels)

# Create a custom collate function
def collate_fn(batch):
    signals = pad_sequence([item[0] for item in batch], batch_first=True)  # Pad sequences
    labels = torch.tensor([item[1] for item in batch], dtype=torch.long)  # Gather labels
    return signals, labels  # Return both signals and labels


# Create DataLoader with the custom collate function
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn,
    drop_last=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=collate_fn,
    drop_last=True
)

In [23]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]  
        out = self.fc(out)
        return out

In [24]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for signals, labels in train_loader:  # This line should work now
            signals = signals.unsqueeze(-1)  # Adding input_size dimension
            outputs = model(signals)

            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [25]:
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for signals, labels in test_loader:
            signals = signals.unsqueeze(-1)
            outputs = model(signals)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')


In [26]:
if __name__ == "__main__":
    input_size = 1  
    hidden_size = 64
    output_size = len(label_encoder.classes_)
    num_layers = 1
    num_epochs = 10
    learning_rate = 0.001

    model = RNN(input_size, hidden_size, output_size, num_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    train_model(model, train_loader, criterion, optimizer, num_epochs)

    # Test the model
    test_model(model, test_loader)

Epoch [1/10], Loss: 0.9140
Epoch [2/10], Loss: 0.9163
Epoch [3/10], Loss: 0.7499
Epoch [4/10], Loss: 0.9919
Epoch [5/10], Loss: 1.0057
Epoch [6/10], Loss: 0.9886
Epoch [7/10], Loss: 1.1056
Epoch [8/10], Loss: 0.9597
Epoch [9/10], Loss: 0.9915
Epoch [10/10], Loss: 0.9966
Accuracy: 60.97%


: 